In [46]:
import pandas as pd

In [47]:
user_raw = pd.read_csv('../dataset/spotify_dataset.csv',on_bad_lines='skip')
song_raw = pd.read_csv('../dataset/spotify_millsongdata.csv')
user_raw.columns = user_raw.columns.str.replace('"', '').str.strip()

In [48]:
song_raw

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...


In [49]:
user_raw

,user_id,artistname,trackname,playlistname
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010
...,...,...,...,...
12891675,2302bf9c64dc63d88a750215ed187f2c,Mötley Crüe,Wild Side,iPhone
12891676,2302bf9c64dc63d88a750215ed187f2c,John Lennon,Woman,iPhone
12891677,2302bf9c64dc63d88a750215ed187f2c,Tom Petty,You Don't Know How It Feels,iPhone
12891678,2302bf9c64dc63d88a750215ed187f2c,Tom Petty,You Wreck Me,iPhone


In [64]:
merged_df = user_raw.merge(song_raw, left_on='trackname', right_on='song', how='left')
merged_df = merged_df.dropna()
merged_df

,user_id,artistname,trackname,playlistname,artist,song,link,text
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010,Elvis Costello,Accidents Will Happen,/e/elvis+costello/accidents+will+happen_200471...,"Oh, I just don't know where to begin \r\nLove..."
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010,Frank Sinatra,Accidents Will Happen,/f/frank+sinatra/accidents+will+happen_2016452...,The likes of you may never be attracted to the...
5,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010,Elvis Costello,Alison,/e/elvis+costello/alison_10100103.html,Oh it's so funny to be seeing you after so lon...
6,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010,Linda Ronstadt,Alison,/l/linda+ronstadt/alison_20241364.html,Well it's so funny to be seeing you after so l...
8,9cc0cfd4d7d7885102480dd99e7a90d6,Paul McCartney,Band On The Run,HARD ROCK 2010,Foo Fighters,Band On The Run,/f/foo+fighters/band+on+the+run_21057136.html,"Shout, shout, let it all out \r\nThese are th..."
...,...,...,...,...,...,...,...,...
19163300,2302bf9c64dc63d88a750215ed187f2c,Stone Temple Pilots,Wicked Garden,iPhone,Stone Temple Pilots,Wicked Garden,/s/stone+temple+pilots/wicked+garden_20132554....,Can you see like a child? \r\nCan you see wha...
19163302,2302bf9c64dc63d88a750215ed187f2c,John Lennon,Woman,iPhone,Free,Woman,/f/free/woman_20344442.html,Carry me away \r\nI know your angel eye can s...
19163303,2302bf9c64dc63d88a750215ed187f2c,John Lennon,Woman,iPhone,Maroon 5,Woman,/m/maroon+5/woman_10175484.html,If I'd be so inclined to climb up beside you ...
19163304,2302bf9c64dc63d88a750215ed187f2c,John Lennon,Woman,iPhone,Ozzy Osbourne,Woman,/o/ozzy+osbourne/woman_20421453.html,"Woman I can hardly express, \r\nMy mixed emot..."
